# Jupyter Notebook for the Color Branch

## Standard Imports

## Code to extract Hex Code from Images

In [ ]:
# State: working
# ToDo: need to check for optimal color for 'images' and code clean up
'''
Imports:
'''
import cv2
import numpy as np
import matplotlib.pyplot as plt
import X_Code.variables as v

'''
Additional Imports:
'''
import os
from collections import Counter
from sklearn.cluster import KMeans
from skimage.color import rgb2lab, deltaE_cie76

'''
Setup:
'''
COLORS = { # Source: http://www.workwithcolor.com/cyan-color-hue-range-01.htm
        'Test': [200,213,48],
        'Bubbles': [231,254,255],
        'Cyan': [0,255,255],
        'Columbia Blue': [155,221,255],
        'Bright Turquoise': [8,232,222],
        'Baby Blue': [137,207,240],
        'Sky Blue': [135,206,235],
        'Pastel Blue': [174,198,207],
        'Turquoise': [48,213,200],
        'Dark Cyan': [0,139,139],
        'Cerulean': [0,123,167],
        'Teal': [0,128,128],
        'Pine Green': [1,121,111],
        'Dark Slate Gray': [47,79,79],					
         }
         
IMAGE_DIRECTORY = 'images_testing' # images path
images = []

def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

for file in os.listdir(IMAGE_DIRECTORY):
    if not file.startswith('.'):
        images.append(get_image(os.path.join(IMAGE_DIRECTORY, file)))

'''
Functions: returns colors Hex Code
'''
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

def get_colors(image, number_of_colors, show_chart):
    
    modified_image = cv2.resize(image, (600, 400), interpolation = cv2.INTER_AREA)
    modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)
    
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)
    
    counts = Counter(labels)
    # sort to ensure correct color percentage
    counts = dict(sorted(counts.items())) 
    
    center_colors = clf.cluster_centers_
    # get ordered colors by iterating through the keys
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
    rgb_colors = [ordered_colors[i] for i in counts.keys()]

    if (show_chart):
        plt.figure(figsize = (8, 6))
        plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)
        plt.show()

    return rgb_colors

def match_image_by_color(image, color, threshold, number_of_colors ): 
    
    image_colors = get_colors(image, number_of_colors, False)
    selected_color = rgb2lab(np.uint8(np.asarray([[color]])))

    select_image = False
    for i in range(number_of_colors):
        curr_color = rgb2lab(np.uint8(np.asarray([[image_colors[i]]])))
        diff = deltaE_cie76(selected_color, curr_color)
        if (diff < threshold):
            select_image = True
    
    return select_image

def show_selected_images(images, color, threshold, colors_to_match):
    index = 1
    
    for i in range(len(images)):
        selected = match_image_by_color(images[i],
                                        color,
                                        threshold,
                                        colors_to_match)
        if (selected):
            images[i] = cv2.cvtColor(images[i],cv2.COLOR_BGR2RGB) # convert again to RGB Color Model because OpenCV uses BGR as Default Model
            cv2.imshow("out",images[i],)
            cv2.waitKey(0)
            index += 1
        else:
            print("Image number {} is clear" .format(i))

'''
Call Functions:
'''
#get_colors(v.image_ext, 6, True) # uncomment for pie chart
show_selected_images(images, COLORS['Cyan'], 55, 15)



## Code to extract RGB Code from Video-Images

In [ ]:
# State: working
# ToDo: check if RGB model values are BGR model values
'''
Imports:
'''
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import X_Code.variables as v

'''
Setup:
'''
img = v.image_int

index=["color", "color_name", "hex", "R", "G", "B"] # index colors
csv = pd.read_csv('colors.csv', names=index, header=None) # and read csv file

'''
Globals:
'''
clicked = False
r = g = b = xpos = ypos = 0

'''
Function: returns RGB Color Code
'''
def recognize_color(R,G,B):
    minimum = 10000
    for i in range(len(csv)):
        d = abs(R- int(csv.loc[i,"R"])) + abs(G- int(csv.loc[i,"G"]))+ abs(B- int(csv.loc[i,"B"])) # Get RGB value trough simple absolute subtraction
        if(d<=minimum):
            minimum = d
            cname = csv.loc[i,"color_name"]
    return cname
'''
Showcase:
'''
def mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global b,g,r,xpos,ypos, clicked
        clicked = True
        xpos = x
        ypos = y
        b,g,r = img[y,x]
        b = int(b)
        g = int(g)
        r = int(r)

cv2.namedWindow('Color Detection')
cv2.setMouseCallback('Color Detection', mouse_click)

while(1):
    cv2.imshow("Color Detection",img)
    if (clicked):
   
        #cv2.rectangle(image, startpoint, endpoint, color, thickness)-1 fills entire rectangle 
        cv2.rectangle(img,(20,20), (750,60), (b,g,r), -1)#Creating text string to display( Color name and RGB values )
        text = recognize_color(r,g,b) + ' R='+ str(r) +  ' G='+ str(g) +  ' B='+ str(b)
        
        #cv2.putText(img,text,start,font(0-7),fontScale,color,thickness,lineType )
        cv2.putText(img, text,(50,50),2,0.8,(255,255,255),2,cv2.LINE_AA)#For very light colours we will display text in black colour
        if(r+g+b>=600):
            cv2.putText(img, text,(50,50),2,0.8,(0,0,0),2,cv2.LINE_AA)
            
        clicked=False

    #Break the loop when user hits 'esc' key    
    if cv2.waitKey(20) & 0xFF ==27:
        break
cv2.destroyAllWindows()

## Code to extract a 3D Visualization of the Colors present in a given Image 

In [ ]:
# State: works
# ToDo: *empty*
'''
Imports:
'''
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import X_Code.variables as var

'''
Additional imports:
'''
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors

'''
Functions: to visualize colors via HSV Color Mode
'''
def rgb_visualize(img):
    '''
    Setup:
    '''
    r, g, b = cv2.split(img)
    fig = plt.figure()
    axis = fig.add_subplot(1, 1, 1, projection="3d")

    '''
    Create Graphic:
    '''
    pixel_colors = img.reshape((np.shape(img)[0]*np.shape(img)[1], 3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    pixel_colors = norm(pixel_colors).tolist()

    axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
    axis.set_xlabel("Red")
    axis.set_ylabel("Green")
    axis.set_zlabel("Blue")
    plt.show()

def hsv_visualize(img):

    '''
    Setup:
    '''
    colors_origin = var.image_ext # to alter the Graphic from the HSV Model back to the original RGB Color Model

    h, s, v = cv2.split(img)
    fig = plt.figure()
    axis = fig.add_subplot(1, 1, 1, projection="3d")

    '''
    Create Graphic:
    '''
    pixel_colors = colors_origin.reshape((np.shape(colors_origin)[0]*np.shape(colors_origin)[1], 3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    pixel_colors = norm(pixel_colors).tolist()

    axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
    axis.set_xlabel("Hue")
    axis.set_ylabel("Saturation")
    axis.set_zlabel("Value")
    plt.show()

'''
Call Functions:
'''
rgb_visualize(var.image_ext)
hsv_visualize(var.image_HSV2)



## Code to manually adjust Image to the HSV Color Model

In [ ]:
# State: works
# ToDo: Code cleanup maybe

'''
Imports:
'''
import cv2
import numpy as np
import X_Code.variables as v

'''
Functions:
'''
def empty(a):
    pass

'''
Setup Trackbars:
'''
cv2.namedWindow("Trackbars")
cv2.resizeWindow("Trackbars",640,240)

cv2.createTrackbar("Hue Min", "Trackbars",0,179,empty) # set arg1 to 0
cv2.createTrackbar("Hue Max", "Trackbars",179,179,empty) # set arg1 to 179
cv2.createTrackbar("Sat Min", "Trackbars",0,255,empty) # set arg1 to 0
cv2.createTrackbar("Sat Max", "Trackbars",255,255,empty) # set arg1 to 255
cv2.createTrackbar("Val Min", "Trackbars",0,255,empty) # set arg1 to 0
cv2.createTrackbar("Val Max", "Trackbars",255,255,empty) # set arg1 to 255

while True:

    h_min = cv2.getTrackbarPos("Hue Min", "Trackbars")
    h_max = cv2.getTrackbarPos("Hue Max", "Trackbars")
    s_min = cv2.getTrackbarPos("Sat Min", "Trackbars")
    s_max = cv2.getTrackbarPos("Sat Max", "Trackbars")
    v_min = cv2.getTrackbarPos("Val Min", "Trackbars")
    v_max = cv2.getTrackbarPos("Val Max", "Trackbars")

    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])

    image_mask = cv2.inRange(v.image_int, lower, upper)
    image_result = cv2.bitwise_and(v.image_int, v.image_int, mask = image_mask)

    # Show images stacked
    images_stacked = v.stackImages(0.6, ([v.image_int, v.image_HSV1], [image_mask, image_result]))
    cv2.imshow("Stacked Images", images_stacked)

    cv2.waitKey(1)